# Performing surgery

We have a model, and want to change just one part of it. How can we do that?

In Feedbax, objects are largely *immutable* once they are constructed. That means we can't simply alter them once they've been created. 

!!! Note     
    Feedbax objects are immutable because they are Equinox modules. It's Equinox that enforces the rules we'll discuss in this example.

Let's start with a pre-built model.

In [2]:
import jax

from feedbax.xabdeef import point_mass_nn_simple_reaches


context = point_mass_nn_simple_reaches(key=jax.random.PRNGKey(0))
model = context.model  # Shorthand

CUDA backend failed to initialize: Unable to load CUDA. Is it installed? (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/mll/.miniforge3/envs/fx/lib/python3.11/site-packages/diffrax/adjoint.py:665: UserWarning: As of Equinox 0.10.7, `equinox.filter_custom_vjp.defvjp` is deprecated in favour of `.def_fwd` and `.def_bwd`. This new API supports symbolic zeros, which allow for more efficient autodifferentiation rules. In particular:
- the fwd and bwd functions take an extra `perturbed` argument, which     indicates which primals actually need a gradient. You can use this     to skip computing the gradient for any unperturbed value. (You can     also safely just ignore this if you wish.)
- `None` was previously passed to indicate a symbolic zero gradient for     all objects that weren't inexact arrays, but all inexact arrays     always had an array-valued gradient. Now, `None` may also be passed     to indicate that an inexact array has a symbolic zero gradient.
  _loop_backsolv

This model has a point mass of mass $1.0$ as its skeleton.

In [3]:
model.step.mechanics.plant.skeleton

PointMass(mass=1.0)

If we try to directly alter the model to use a point mass of mass $5.0$, an error is raised.

In [4]:
from feedbax.mechanics.skeleton import PointMass 

# Try to replace the entire point mass
model.step.mechanics.plant.skeleton = PointMass(5.0)

FrozenInstanceError: cannot assign to field 'skeleton'

In [ ]:
# Or just try to change the mass
model.step.mechanics.plant.skeleton.mass = 5.0

FrozenInstanceError: cannot assign to field 'mass'

This kind of direct re-assignment is common in Python. It might seem inconvenient to have it outlawed! 

Well, it is still possible to alter our model. But if we want to switch out just the point mass, we have to do something slightly more complex.

In [6]:
import equinox as eqx

model_heavy = eqx.tree_at(
    lambda m: m.step.mechanics.plant.skeleton, 
    model, 
    PointMass(5.0)
)

To replace a part of our model tree, we use the `tree_at` function provided by Equinox. 

The use of `lambda` in the first agument to `tree_at` is similar to the `lambda` we used in Example 1. There, we defined a function `where_train` that picked out which parts of the model should be trainable. Here, our function picks out which part of our model will be replaced.

The second argument is just `model`, which is the model we want to alter. 

The third argument is the part we want to replace it with.

Why the added complexity? *It forces us never to alter our models in-place*. The function `eqx.tree_at` does not modify `model` directly, but *returns a copy* of `model` which possesses the alterations. Here, we assign the new object to `model_heavy`, since maybe we want to be able to refer to both the original model (still called `model`!) and the altered one. But we could just as easily have written

In [ ]:
model = eqx.tree_at(
    lambda m: m.step.mechanics.plant.skeleton, 
    model, 
    PointMass(5.0)
)

This means "create an altered model, and make it so that `model` now refers to the new model---I don't need to refer to the original one by that name anymore".

There are downstream advantages to our ban. The downside to in-place changes is that it can be hard to keep track of their hidden consequences. Some parts of my code may depend on a given object, and if other parts of my code can reach into that object and make implicit alterations, the relationships within my code may be altered without consent of all the stakeholders, so to speak.

When we are forced to *return* an altered object rather than *mutate* an existing object, the consequences are always out in the open. We have to be explicit about what-refers-to-what, with respect to what-has-changed.